In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.patches as mpatches
%matplotlib inline

In [ ]:
!ls ../d4/datasets/

In [ ]:
path = '../d4/datasets'
shops = pd.read_csv(path + "/shops-translated.csv")
items = pd.read_csv(path + "/items-translated.csv")
item_categories = pd.read_csv(path + "/item_categories-translated.csv")
items_russian = pd.read_csv(path + "/items_russian.csv")
train = pd.read_csv(path + "/sales_train.csv")
test = pd.read_csv(path + "/test.csv")

In [ ]:
item_categories = item_categories["item_category_id,item_category_name_translated"].str.split(',', expand=True)
item_categories.columns = ["item_category_id", "item_category_name", "c", "d"]
item_categories = item_categories.drop(["c", "d"], axis=1)
item_categories

In [ ]:
item_categories["item_type"] = np.nan

In [ ]:
item_categories["item_type"] = item_categories["item_category_name"].str.partition('-')


In [ ]:
item_categories["jnn"] = item_categories["item_category_name"].str.split('-')

In [ ]:
hoo = []
for thingy in item_categories["jnn"]:
    hoo.append(thingy[-1].strip())

In [ ]:
item_categories["item_specified"] = hoo

In [ ]:
item_categories = item_categories.drop(["item_category_name", "jnn"], axis=1)

In [ ]:
item_categories['item_specified'] = np.where(item_categories.item_type == item_categories.item_specified, np.nan, item_categories.item_specified)

In [ ]:
item_categories

In [ ]:
shops

In [ ]:
shops = shops["shop_id,shop_name_translated"].str.partition(',')
shops.columns = ["shop_id", "a", "shop_name"]
shops["shop_id"] = shops["shop_id"].astype(int)

In [ ]:
shop_name_list = shops["shop_name"].tolist()
shop_names = []
places = []
for word in shop_name_list:
    places.append(word.split(' ',1)[0].strip('"'))
    shop_names.append(word.strip('"'))

In [ ]:
shops["location"] = places
shops["shop_name"] = shop_names

In [ ]:
shops = shops.drop("a", axis=1)

In [ ]:
shops

In [ ]:
items.columns = ["item_id", "item_name"]
items

In [ ]:
train[:10]

In [ ]:
T1 = pd.merge(train, items, on="item_id", how='right')
T1["shop_id"] = T1["shop_id"].fillna(999)
T1["shop_id"] = T1["shop_id"].astype(int)

In [ ]:
T1["shop_id"] = T1["shop_id"].astype(int)

In [ ]:
train = pd.merge(T1, shops, on='shop_id', how='outer')
train[:10]

In [ ]:
item_categories["item_category_id"] = item_categories["item_category_id"].astype(float)
items_russian["item_category_id"] = items_russian["item_category_id"].astype(float)
T2 = pd.merge(train, items_russian, on='item_id', how='outer')
T3 = pd.merge(T2, item_categories, on='item_category_id', how='outer')
train = T3.drop("item_name_y", axis=1)
train

In [ ]:
train['date'] = pd.to_datetime(train['date'])

In [ ]:
train['year'] = pd.DatetimeIndex(train['date']).year
train['month'] = pd.DatetimeIndex(train['date']).month

In [ ]:
train.columns = ["date", "date_block", "shop_id", "item_id", "item_price", "item_count_day", "item_name", "shop_name", "location", "item_category_id", "item_type", "item_specified"]
train = train.reindex(columns= ["date_block", "date", "shop_id", "location", "shop_name", "item_category_id", "item_type", "item_specified", "item_id", "item_name", "item_price", "item_count_day"])

In [ ]:
train[:10]

In [ ]:
null_rows = train[train.isna().any(axis=1)]
print("total null rows: ", len(null_rows.index))
print("\ntrain rows with null:\n", train.isna().sum())

In [ ]:
train = train.dropna(thresh=7, axis=0)

In [ ]:
train.isnull().sum()

In [ ]:
train["date_block"].unique()

In [ ]:
train

In [ ]:
train["shop_name"].unique()

In [ ]:
train.dtypes

In [ ]:
plt.figure(figsize=(10,5))
plt.xlim(train.item_price.min(), train.item_price.max()*1.1)
sns.boxplot(x=train.item_price)

plt.figure(figsize=(10,5))
plt.xlim(train.item_count_day.min(), train.item_count_day.max()*1.1)
sns.boxplot(x=train.item_count_day)

In [ ]:
train = train[train.item_price<100000]
train = train[train.item_count_day<2000]

In [ ]:
fig = plt.figure(figsize=(20,20))
ax = fig.add_subplot(111, projection='3d')
xs=train["item_category_id"]
zs=train["item_price"]
ys=train["item_count_day"]
train_plot = ax.scatter(xs, ys, zs, c=xs)
ax.set_xlabel("item category")
ax.set_ylabel("item count")
ax.set_zlabel("item price")
plt.title("Items by category: \n number sold and price")

purple_patch = mpatches.Patch(color='purple', label='Books - Artbook')
darkpurple_patch = mpatches.Patch(color='rebeccapurple', label='Books - Comics')
teal_patch = mpatches.Patch(color='teal', label='Gifts - Attributes')
seagreen_patch = mpatches.Patch(color='lightseagreen', label='Gifts - Soft Toys')
aquamarine_patch = mpatches.Patch(color='mediumaquamarine', label='Gifts - Board Games')
lightgreen_patch = mpatches.Patch(color='lightgreen', label='Gifts - Souvenirs')
yellow_patch = mpatches.Patch(color='yellow', label='Elements of a food')
plt.legend(bbox_to_anchor=(0, 0, 0, 1), handles=[purple_patch, darkpurple_patch, teal_patch, seagreen_patch, aquamarine_patch, lightgreen_patch, yellow_patch])

In [ ]:
fig = plt.figure(figsize=(20,20))
ax = fig.add_subplot(111, projection='3d')
xs=train["item_id"]
zs=train["item_price"]
ys=train["item_count_day"]
item_plot = ax.scatter(xs, ys, zs, c=xs)
ax.set_xlabel("item category")
ax.set_ylabel("item count")
ax.set_zlabel("item price")
plt.title("Items: \n number sold and price")

In [ ]:
train

In [ ]:
T5 = pd.merge(test, train, on=["shop_id", "item_id"], how="left")

In [ ]:
T5

In [ ]:
T5.isnull().sum()

In [ ]:
from matplotlib_venn import venn2

In [ ]:
venn2([set(train.item_id.unique()), set(T5.item_id.unique())], set_labels = ("Train items", "Test items"))
plt.show()

In [ ]:
T5

Find missing item prices:

In [ ]:
item_prices = T5.groupby(["item_id"])["item_price"].mean()
item_prices

In [ ]:
T5.loc[T5['item_price'].isnull(),'item_price'] = T5['item_id'].map(item_prices)

In [ ]:
T5["item_price"].isnull().sum()

In [ ]:
general_price_mean = T5["item_price"].mean()
T5["item_price"] = T5["item_price"].fillna(general_price_mean)

In [ ]:
T5["item_price"].isnull().sum()

Fill missing count:

In [ ]:
T5["item_count_day"].isnull().sum()

In [ ]:
T5["item_count_day"] = T5["item_count_day"].fillna(0)

In [ ]:
T5["item_count_day"].isnull().sum()

Fill missing shop names:

In [ ]:
T5["shop_name"].isnull().sum()

In [ ]:
T5['shop_name'] = T5['shop_name'].astype('category') 

In [ ]:
shops_names = T5.groupby(["shop_id"])["shop_name"].value_counts()
shops_names = pd.DataFrame(shops_names)
shops_names.columns = ["a"]
shops_names

In [ ]:
T5.loc[T5['shop_name'].isnull(),'shop_name'] = T5['shop_id'].map(shops_names["shop_name"])

In [ ]:
T5[T5["shop_name"].isnull()]

In [ ]:
T5

In [ ]:
item_30_test = T5[(T5['item_id'] == 30)]
item_30_test

In [ ]:
T5["shop_name"].unique()